### Aula 13 -  Revisão

Nesta aula iremos **revisar SQL** e fixar os conteúdos aprendidos nas últimas aulas.

## Como resolver os exercícios?

Indicamos que resolva em sua máquina. Utilize o MySQL Workbench ou o conector para testar as queries. Quando estiver bastante certo de que a resposta está correta, anexe no notebook a resposta / imagem de resposta (quando solicitado imagem).

## Import das bibliotecas

Vamos importar as bibliotecas.

In [11]:
import mysql.connector
from functools import partial
import os
from dotenv import load_dotenv

E vamos criar nosso HELPER de conexão com o banco! Perceba que, uma vez configurado o `.env` não precisaremos mais informar usuários, senhas e URLs!

In [26]:
from functools import partial
import mysql.connector

connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='megadados',
    database='XPTO',
    sql_mode='only_full_group_by',
)


def run_db_query(connection, query, args=None):
    with connection.cursor() as cursor:
        print('Executando query:')
        cursor.execute(query, args)
        for result in cursor:
            print(result)


db = partial(run_db_query, connection)

## Exercícios de revisão - Megadados

Uma pizzaria quer informatizar seu cardápio para controlar melhor as pizzas que oferece. O diagrama do modelo relacional mostrado abaixo demonstra a estrutura da informação que se deseja armazenar e consultar:

<img src="img/modelo_relacional.png">

- Tabela “tipo”: armazena os nomes dos tipos de ingrediente de pizza: vegano, sem gluten, sem lactose, vegetariano, etc.
  - Campo “tipo”: o nome do tipo
- Tabela “ingrediente”: armazena informações à respeito dos ingredientes de pizza.
  - Campo “ingrediente”: o nome do ingrediente
  - Campo “preco_unitario”: o custo do ingrediente por unidade
- Tabela “ingredientetipo”: relaciona os ingredientes de pizza com seus tipos. Por exemplo: brócoli é vegano, vegetariano, sem lactose e sem glutem, logo aparece quatro vezes nesta tabela.
  - Campo “ingrediente”: chave estrangeira para a tabela “ingrediente”
  - Campo “tipo”: chave estrangeira para a tabela “tipo”
- Tabela “pizza”: O cardápio da pizzaria
  - Campo “pizza”: o nome da pizza
  - Campo “preco”: o valor de venda da pizza
- Tabela “ingredientepizza”: mostra como cada pizza é feita
  - Campo “ingrediente”: chave estrangeira para a tabela de ingredientes
  - Campo “pizza”: chave estrangeira para a tabela de pizzas
  - Campo “quantidade”: quantas unidades deste ingrediente fazem parte desta pizza

O script `script_001.sql` contem a DDL para criação do banco de dados e também a DML para inserção de alguns valores de exemplo, para ajudar vocês a responder as questões da prova. Se quiser, adicione exemplos à vontade. Nas perguntas a seguir construa código SQL para cumprir o que se pede.

**Exercício 1**: Liste as 3 pizzas que dão mais lucro. O lucro de uma pizza é o seu preço de venda menos o custo total dos ingredientes nela utilizados (não se esqueça de que cada ingrediente da pizza tem uma especificação de quantidade a ser utilizada).

In [51]:
sql_ex1 = '''
SELECT
    p.pizza, 
    (p.preco - (ip.quantidade * i.preco_unitario) ) AS lucro
FROM 
    pizza p INNER JOIN ingredientePizza ip ON p.pizza = ip.pizza
    INNER JOIN ingrediente i on ip.ingrediente = i.ingrediente
    

ORDER BY lucro DESC

LIMIT 3
    
'''
db(sql_ex1)

Executando query:
('incoerente', 193.0)
('incoerente', 190.0)
('incoerente', 180.0)


**Exercício 2**: Construa uma tabela temporária que liste todas as pizzas e a quantidade de ingredientes nela.

In [76]:
sql_ex2 = '''

    SELECT 
        p.pizza,
        COUNT(ip.quantidade) AS quantidade

    FROM 
        pizza p INNER JOIN ingredientePizza ip ON p.pizza = ip.pizza


    GROUP BY
        p.pizza
'''

db(sql_ex2)

Executando query:
('calabresa', 2)
('incoerente', 4)
('legumes', 2)
('mussarela', 2)
('pão', 1)


**Exercício 3**: Construa uma tabela temporária que liste todas as pizzas a quantidade de ingredientes veganos nela (ou seja, onde o tipo é “vegana”).

In [81]:
sql_ex3 = '''
SELECT 
    p.pizza, COUNT(ip.quantidade) AS quantidade
FROM 
    pizza p INNER JOIN ingredientePizza ip ON p.pizza = ip.pizza
    INNER JOIN ingrediente i ON ip.ingrediente = i.ingrediente
    INNER JOIN ingredienteTipo it ON i.ingrediente =  it.ingrediente
    INNER JOIN tipo t ON it.tipo = t.tipo

WHERE
    t.tipo = "vegana"


GROUP BY

    p.pizza


'''
db(sql_ex3)

Executando query:
('legumes', 2)
('incoerente', 2)


**Exercício 4**: Usando as tabelas temporárias dos itens (b) e (c) – mesmo que você não os tenha feito – liste as pizzas veganas da pizzaria. Uma pizza é vegana se todos os seus ingredientes são veganos.

In [20]:
sql_ex4 = '''
SELECT p.pizza
    from pizza p join ingredientePizza ip on p.pizza = ip.pizza
    join ingrediente i on ip.ingrediente = i.ingrediente
    join ingredienteTipo it on i.ingrediente = it.ingrediente
    join tipo t on it.tipo = t.tipo
    where t.tipo = 'vegana'
'''

In [21]:
db(sql_ex4)

Executando query:
('legumes',)
('incoerente',)
('legumes',)
('incoerente',)


**Exercício 5**: Liste os ingredientes e seus tipos, um ingrediente por linha (Dica: use a função GROUP_CONCAT). O resultado deve ser da seguinte forma:

<img src="img/group_concat.png">


In [22]:
sql_ex5 = '''
SELECT i.ingrediente, GROUP_CONCAT(t.tipo)
from pizza p join ingredientePizza ip on p.pizza = ip.pizza
join ingrediente i on ip.ingrediente = i.ingrediente
join ingredienteTipo it on i.ingrediente = it.ingrediente
join tipo t on it.tipo = t.tipo
GROUP BY i.ingrediente
'''

In [23]:
db(sql_ex5)

Executando query:
('abobrinha', 'sem gluten,sem lactose,vegana,vegetariana')
('calabresa', 'sem gluten,sem lactose')
('massa', 'vegetariana,vegetariana,vegetariana')
('massa sem gluten', 'sem gluten,sem lactose,vegana,vegetariana,sem gluten,sem lactose,vegana,vegetariana')
('mussarela', 'sem gluten,vegetariana,sem gluten,vegetariana')
('mussarela vegana', 'sem gluten,sem lactose,vegana,vegetariana')
('picanha', 'sem gluten,sem lactose')


**Exercício 6**: Construa o diagrama do modelo entidade-relacionamento correspondente ao diagrama do modelo relacional acima. Normalmente a ordem de construção das coisas é o contrário disso, mas aqui eu estou apenas explorando o conhecimento de vocês! Gere um arquivo PNG, JPG ou PDF com a figura resultante, deixe na pasta `img`, e altere a tag de imagem.

<div class="alert alert-success">
Edite na resposta o caminho para a imagem!
    
<img src="img/exemplo.png">

</div>